In [1]:
import torch
from torch.nn import functional as F
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, DistributedSampler
import pytorch_lightning as pl
import numpy as np
from torchvision import datasets, transforms
import os
from tqdm import tqdm

# Pytorch

In [2]:
BATCH_SIZE = 512
N_EPOCHS = 5
LEARNING_RATE = 1e-3

Давайте загрузм данные и передадим в dataset пайплайн предобработки

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

train_dataset = datasets.FashionMNIST(os.getcwd(), train=True, download=False,
                   transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=10)

In [4]:
def accuracy(true, pred):
    return sum(pred == true) / len(pred)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
    
class PytorchModel(torch.nn.Module):
    def __init__(self):
        super(PytorchModel, self).__init__()
        self.sequential_module = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            Flatten(),

            nn.Linear(1152, 64),
            nn.ReLU(inplace=False),
            nn.Linear(64, 32),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.5),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.sequential_module(x)
        x = F.log_softmax(x, dim=-1)
        return x

In [5]:
class TrainModel():
    
    def __init__(self, device='cpu', dp=False):
        self.learning_rate = LEARNING_RATE
        self.n_epochs = N_EPOCHS
        self.device = device
        self.model = PytorchModel()
        if self.device != 'cpu':
            self.model.to(self.device)
            if dp:
                self.model = nn.DataParallel(self.model)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        
    def run_train(self, train_loader):
        all_preds = []
        all_facts = []
        
        self.criterion = nn.NLLLoss()
        for epoch in range(self.n_epochs):
            self.model.train()
            for x, y in tqdm(train_loader):
                
                if self.device != 'cpu':
                    x = x.to(self.device)
                    y = y.to(self.device)
                
                self.model.zero_grad()
                y_pred = self.model(x)
                
                loss = self.criterion(y_pred, y)
                loss.backward()
                
                y_pred = y_pred.argmax(dim=1, keepdim=True)
                y_pred = torch.squeeze(y_pred)
                
                all_preds.extend(y_pred.cpu().detach().numpy())
                all_facts.extend(y.cpu().detach().numpy())
                
                self.optimizer.step()
                
            print(f'epoch = {epoch}, accuracy = {accuracy(np.array(all_preds), np.array(all_facts))}')
        return self

### cpu

In [6]:
%%time

pytorch_model = TrainModel()
pytorch_model = pytorch_model.run_train(train_loader)

100%|██████████| 118/118 [01:44<00:00,  1.13it/s]


epoch = 0, accuracy = 0.7737833333333334


100%|██████████| 118/118 [01:39<00:00,  1.18it/s]


epoch = 1, accuracy = 0.8589416666666667


100%|██████████| 118/118 [01:45<00:00,  1.12it/s]


epoch = 2, accuracy = 0.8919777777777778


100%|██████████| 118/118 [01:44<00:00,  1.13it/s]


epoch = 3, accuracy = 0.910025


100%|██████████| 118/118 [01:46<00:00,  1.11it/s]


epoch = 4, accuracy = 0.9215733333333334
CPU times: user 58min 17s, sys: 4min 59s, total: 1h 3min 16s
Wall time: 8min 42s


### 1 gpu

In [7]:
%%time

pytorch_model = TrainModel(device='cuda:0')
pytorch_model = pytorch_model.run_train(train_loader)

100%|██████████| 118/118 [00:05<00:00, 20.15it/s]


epoch = 0, accuracy = 0.8025666666666667


100%|██████████| 118/118 [00:03<00:00, 34.61it/s]


epoch = 1, accuracy = 0.8743583333333333


100%|██████████| 118/118 [00:03<00:00, 34.78it/s]


epoch = 2, accuracy = 0.9037277777777778


100%|██████████| 118/118 [00:03<00:00, 34.85it/s]


epoch = 3, accuracy = 0.9204208333333334


100%|██████████| 118/118 [00:03<00:00, 34.72it/s]


epoch = 4, accuracy = 0.9311066666666666
CPU times: user 9.27 s, sys: 5.84 s, total: 15.1 s
Wall time: 22.8 s


### multi-gpu

In [8]:
%%time

pytorch_model = TrainModel(device='cuda:0', dp=True)
pytorch_model = pytorch_model.run_train(train_loader)

100%|██████████| 118/118 [00:08<00:00, 14.61it/s]


epoch = 0, accuracy = 0.779


100%|██████████| 118/118 [00:03<00:00, 30.36it/s]


epoch = 1, accuracy = 0.85905


100%|██████████| 118/118 [00:03<00:00, 31.75it/s]


epoch = 2, accuracy = 0.8908611111111111


100%|██████████| 118/118 [00:03<00:00, 32.16it/s]


epoch = 3, accuracy = 0.90885


100%|██████████| 118/118 [00:03<00:00, 32.45it/s]


epoch = 4, accuracy = 0.9202933333333333
CPU times: user 15.7 s, sys: 6.49 s, total: 22.2 s
Wall time: 25 s


### torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)

device_ids = [0, 1]

Создадим модель при помощи pytorch-lightening. У него много возможностей, в том числе легкое логирование всего в tensorboard, уже готовый конструктор для обучения моделей

In [5]:
def accuracy(true, pred):
    return sum(pred == true) / len(pred)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class LightModel(pl.LightningModule):
    def __init__(self, learning_rate):
        super(LightModel, self).__init__()
        self.sequential_module = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            Flatten(),

            nn.Linear(1152, 64),
            nn.ReLU(inplace=False),
            nn.Linear(64, 32),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.5),
            nn.Linear(32, 10)
        )
        self.criterion = nn.NLLLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        x = self.sequential_module(x)
        x = F.log_softmax(x, dim=-1)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        pred = torch.squeeze(pred)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss, 'accuracy': accuracy(pred, y)}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

Trainer чем-то похож на обучение моделей в keras. gpus=None говорит о том, что обучать мы будем все на CPU

In [17]:
trainer_cpu = pl.Trainer(accelerator='cpu', max_epochs=N_EPOCHS)
nn_model = LightModel(LEARNING_RATE)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/app-root/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(


In [18]:
%time trainer_cpu.fit(nn_model, train_loader)


  | Name              | Type       | Params
-------------------------------------------------
0 | sequential_module | Sequential | 358 K 
1 | criterion         | NLLLoss    | 0     
-------------------------------------------------
358 K     Trainable params
0         Non-trainable params
358 K     Total params
1.433     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 1h 8min 35s, sys: 13min 53s, total: 1h 22min 29s
Wall time: 11min 2s


Предыдущий вариант был долгим...Давайте возьмем 1 гпу (если написать devices = [0], то выберется с индексом 0).

In [19]:
trainer_gpu = pl.Trainer(accelerator='gpu', max_epochs=N_EPOCHS, devices=1)
nn_model = LightModel(LEARNING_RATE)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
%time trainer_gpu.fit(nn_model, train_loader)

You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name              | Type       | Params
-------------------------------------------------
0 | sequential_module | Sequential | 358 K 
1 | criterion         | NLLLoss    | 0     
-------------------------------------------------
358 K     Trainable params
0         Non-trainable params
358 K     Total params
1.433     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 10.5 s, sys: 6.1 s, total: 16.6 s
Wall time: 22.4 s


Намного лучше. А что если взять 2 гпу? Для этого нужно немного переписать код, так как мы будем использовать стратегию dp - батч делится на равные части между всеми задействованными gpu

GPU можно выбирать, либо сказать сколько хотим взять и будут выбраны наименее загруженные

In [6]:
trainer_gpu = pl.Trainer(accelerator='auto', max_epochs=5, devices=2,
                         log_every_n_steps=10, use_distributed_sampler=False)#gpus=[0, 1]
nn_model = LightModel(LEARNING_RATE)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
%time trainer_gpu.fit(nn_model, train_loader)

Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
[W socket.cpp:426] [c10d] The server socket cannot be initialized on [::]:41665 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:601] [c10d] The client socket cannot be initialized to connect to [localhost]:41665 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:601] [c10d] The client socket cannot be initialized to connect to [localhost]:41665 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:601] [c10d] The client socket cannot be initialized to connect to [localhost]:41665 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:601] [c10d] The client socket cannot be initialized to connect to [localhost]:41665 (errno: 97 - Address family not supported by protocol).
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distribute

Training: 0it [00:00, ?it/s]

/opt/app-root/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('train_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(
`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 1.39 s, sys: 785 ms, total: 2.18 s
Wall time: 50.6 s


In [9]:
torch.cuda.empty_cache()

# Домашнее задание \ проект

В данном проекте вам нужно будет провести стандартную работу - загрузить данные, поисследовать их, сделать модельку. Сразу оговорюсь, что подбор параметров, интересные архитектуры...на это я внимания обращать не буду, у нас курс не про ML. Главное - это использования GPU. Что нужно?

    1) Загрузить данны при помощи cudf\dask-cudf
    2) Произвести базовый анализ данных (распределение классов, средние значения фичей по классам и вот это вот все). Любые преобразования данных на ваше усмотрение (только комментируйте=) )
    3) Попробуйте в ходе анализа \ обработки найти применение cuda.jit и cupy (достаточно по 1 примеру, даже искуственному)
    4) Взять два метода снижения размерности из cuml (на лекции был UMAP, еще есть tSNE, tSVD, PCA...), обучить их и визуализировать наши классы в пространстве размерности 2 (раскрасить классы будет хорошей идеей)
    5) Обучите нейронку на GPU\Multi-GPU, а также какую-нибудь еще модельку на GPU\Multi-GPU.

Если в пункте 5 будет Multi-GPU - это дополнительные баллы, а также если еще добавите, например, метод кластеризации просто для сравнения =)

P.S. прошу в коде давать комментарии того, что делаете. Пункты 4 и 5 также зачту за домашку 3 занятия.

Данные находятся в файле Train_Set_90621.csv
Amount Defaulted - эту переменную нужно удалить=)

И все на GPU, никаких любимых pandas, numpy и прочего)) Думаю, из описания понятно, как я буду оценивать работу)

Цель всего этого - чтобы вы побольше потыкали GPU